In [1]:
## installing packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q graphviz --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.6.1 requires tokenizers<0.11,>=0.10.1, but you have tokenizers 0.9.4 which is incompatible.
pathos 0.2.8 requires dill>=0.3.4, but you have dill 0.3.3 which is incompatible.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mxnet 1.8.0.post0 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.17 which is incompatible.
autogluon-vision 0.2.0 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.17 which is incompatible.
autogluon-extra 0.2.0 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.17 which is incompatible.
autogluon-core 0.2.0 requires graphviz<0.9.0,>=0.8.1, but

In [2]:
# imports
import gc
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-aug-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-aug-2021/test.csv'

time_limit = 20000 # (in secs. for train run: set as 300 which gave public score ~10)

In [4]:
train = pd.read_csv(PATH_TRAIN, index_col="id")
test = pd.read_csv(PATH_TEST, index_col="id")

In [5]:
target_loss = train.loss
train.drop(['loss'], axis=1, inplace=True)

In [6]:
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,0.376622,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,0.147973,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,0.335985,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,0.336490,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,0.528518,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197


In [7]:
## imports
from autogluon.tabular import TabularPredictor, TabularDataset

In [8]:
## run model for carbon monoxide
train['target'] = target_loss

model_ag = TabularPredictor(problem_type='regression', label='target')

model_ag.fit(train_data=train, 
              time_limit=time_limit, 
              presets='best_quality',
              num_stack_levels = 3,
              num_bag_folds = 5,
              num_bag_sets = 1,)

del train['target']

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[1000]	train_set's rmse: 7.38773	valid_set's rmse: 7.87666
[1000]	train_set's rmse: 7.39742	valid_set's rmse: 7.85692


In [9]:
## check leaderboard for carbon monoxide
model_ag.leaderboard()

                     model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3  -7.846861      61.532278   6667.427154                0.018432          13.034587            3       True         15
1        LightGBMXT_BAG_L2  -7.847510      60.507628   6565.320123                1.108506          51.804541            2       True         10
2      WeightedEnsemble_L4  -7.847693     143.882922  16938.391557                0.017395          11.298949            4       True         23
3          CatBoost_BAG_L2  -7.847942      59.526059   6557.930536                0.126938          44.414953            2       True         13
4        LightGBMXT_BAG_L3  -7.848289     121.656844  12410.429888                1.162339          49.087738            3       True         18
5          LightGBM_BAG_L2  -7.848323      60.278402   6558.173072                0.879280          44.657490            2       T

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-7.846861,61.532278,6667.427154,0.018432,13.034587,3,True,15
1,LightGBMXT_BAG_L2,-7.847510,60.507628,6565.320123,1.108506,51.804541,2,True,10
2,WeightedEnsemble_L4,-7.847693,143.882922,16938.391557,0.017395,11.298949,4,True,23
3,CatBoost_BAG_L2,-7.847942,59.526059,6557.930536,0.126938,44.414953,2,True,13
4,LightGBMXT_BAG_L3,-7.848289,121.656844,12410.429888,1.162339,49.087738,3,True,18
5,LightGBM_BAG_L2,-7.848323,60.278402,6558.173072,0.879280,44.657490,2,True,11
6,CatBoost_BAG_L3,-7.848512,120.652200,12407.635092,0.157695,46.292942,3,True,21
7,WeightedEnsemble_L2,-7.848684,8.376316,324.226964,0.017038,10.045665,2,True,7
8,WeightedEnsemble_L5,-7.849457,176.055661,17305.157550,0.014892,9.547093,5,True,30
9,LightGBM_BAG_L3,-7.849773,121.250526,12402.407727,0.756021,41.065577,3,True,19


In [10]:
preds_autogluon = model_ag.predict(TabularDataset(test))

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
## create submission
submission = pd.DataFrame({
    'id': preds_autogluon.index,
    'loss': preds_autogluon
})

submission.head()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,loss
id,,
250000,250000,7.679582
250001,250001,4.746612
250002,250002,7.943443
250003,250003,7.402416
250004,250004,7.621213


In [12]:
## save submission
PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon_' + str(time_limit) + '.csv'
PATH_AUTOGLUON_SUBMISSION

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'submission_autogluon_20000.csv'

In [13]:
submission.to_csv(PATH_AUTOGLUON_SUBMISSION, index=False)

In [14]:
## clear memory
shutil.rmtree('AutogluonModels')

gc.collect()

1021